# Experiment 3 Results

In [1]:
from visualization.utils import make_defaults
from visualization.utils import defaults as d
from helpers.paths import Path
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

make_defaults()

In [2]:
def columns_to_rows(df_start):
    df = df_start.melt(id_vars='eval/step', var_name='id_var', value_name='value')
    df[['id', 'var']] = df['id_var'].str.split(' - ', expand=True)
    df = df.pivot_table(index=['eval/step', 'id'], columns='var', values='value').reset_index()
    return df

In [15]:
df_sac_summary = pd.read_csv(Path().data / 'exp3_fault_fp41w198_summary.csv')
# df_sac_eval = columns_to_rows(pd.read_csv(Path().data / 'exp3_sac_elevator_online.csv'))

## Summary results

In [19]:
df_sac_summary[['eval/nmae', 'online/nmae', 'hybrid_improvement', 'sac_model']].groupby('sac_model').describe()

eval/nmae                                     \
                                  count      mean  std       min       25%   
sac_model                                                                    
SAC-citation/denim-leaf-172         5.0  0.143081  0.0  0.143081  0.143081   
SAC-citation/divine-grass-171       5.0  0.214873  0.0  0.214873  0.214873   
SAC-citation/firm-feather-173       5.0  0.195814  0.0  0.195814  0.195814   

                                                            online/nmae  \
                                    50%       75%       max       count   
sac_model                                                                 
SAC-citation/denim-leaf-172    0.143081  0.143081  0.143081         5.0   
SAC-citation/divine-grass-171  0.214873  0.214873  0.214873         5.0   
SAC-citation/firm-feather-173  0.195814  0.195814  0.195814         5.0   

                                         ...                      \
                                   mean  ...       75%       max   
sac_model                                ...                       
SAC-citation/denim-leaf-172    0.046493  ...  0.048199  0.056684   
SAC-citation/divine-grass-171  0.031677  ...  0.031350  0.034756   
SAC-citation/firm-feather-173  0.045782  ...  0.048538  0.058308   

                              hybrid_improvement                       \
                                           count       mean       std   
sac_model                                                               
SAC-citation/denim-leaf-172                  5.0   9.658849  0.662112   
SAC-citation/divine-grass-171                5.0  18.319511  0.176756   
SAC-citation/firm-feather-173                5.0  15.003214  0.807081   

                                                                           \
                                     min        25%        50%        75%   
sac_model                                                                   
SAC-citation/denim-leaf-172     8.639737   9.488199   9.805916   9.934609   
SAC-citation/divine-grass-171  18.011622  18.352226  18.366518  18.412788   
SAC-citation/firm-feather-173  13.750628  14.727602  15.199184  15.537486   

                                          
                                     max  
sac_model                                 
SAC-citation/denim-leaf-172    10.425783  
SAC-citation/divine-grass-171  18.454398  
SAC-citation/firm-feather-173  15.801171  

[3 rows x 24 columns]

## Run results

In [23]:
df_light = df_sac_eval[df_sac_eval['eval/step'] % 5 == 0]

In [29]:
# fig, ax = plt.subplots(1, 3)

sns.lineplot(data=df_light, x="eval/step", y="eval/theta", ax=ax[0])
# sns.lineplot(data=df_light, x="eval/step", y="eval/phi", ax=ax[1])
# sns.lineplot(data=df_light, x="eval/step", y="eval/beta", ax=ax[2])
plt.show()

In [86]:
fig_layout = dict(
    autosize=False,  # Disable auto size
    width=d.width,  # width in pixels
    height=300,  # height in pixels
    font=d.font,
    margin=dict(  # Reducing the white space around the plot
        l=40,  # left margin
        r=2,  # right margin
        b=10,  # bottom margin
        t=45,  # top margin
    )
)

In [87]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming df is your dataframe, 'time' is x-axis, 'signal1' and 'signal2' are y-values.
fig = make_subplots(rows=1, cols=2)

df_numeric = df_sac_eval.apply(pd.to_numeric, errors='coerce')
new_df = df_numeric.groupby(["eval/step"]).agg(['mean', 'std']).reset_index()
new_df["eval/theta", "std"]
new_df["eval/step"]*=0.01

fig.add_trace(
    go.Scatter(x=new_df["eval/step"], y=new_df["eval/theta", "mean"], name='theta'),
    row=1, col=1
)


fig.add_trace(
   go.Scatter(
        name='theta_ref',
        x=new_df["eval/step"],
        y=new_df["eval/theta_ref", "mean"],
        mode='lines',
       line=dict(dash='dash')
    ),
    row = 1, col = 1
)

fig.add_trace(
    go.Scatter(
        name='Upper Bound',
        x=new_df["eval/step"],
        y=new_df["eval/theta", "mean"] + new_df["eval/theta", "std"],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False,
    ),
    row = 1, col = 1
)

fig.add_trace(
    go.Scatter(
        name='Lower Bound',
        x=new_df["eval/step"],
        y=new_df["eval/theta", "mean"] - new_df["eval/theta", "std"],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        fillcolor='rgba(68, 68, 68, 0.2)',
        fill='tonexty',
        showlegend=False,
    ),
    row = 1, col = 1
)

fig.update_layout(**fig_layout)
# fig.show()

style(font={'family': 'Fira Sans, Helvetica, Arial, sans-serif', 'color': '#333333', 'size': 20}, width=500)